In [10]:
"""
@File    : edu_airdrop_v2.ipynb
@Create  : 2024/10/01 16:46:43
@Modify  : 2024/10/01 16:46:43
@Author  : Flora Sun
@Contact : floras@animocabrands.com
@Desc    : None
"""


'\n@File    : edu_airdrop_v2.ipynb\n@Create  : 2024/10/01 16:46:43\n@Modify  : 2024/10/01 16:46:43\n@Author  : Flora Sun\n@Contact : floras@animocabrands.com\n@Desc    : None\n'

# Import libraries

In [2]:
import os
import numpy as np
import pandas as pd
import sybil_functions
import importlib
from datetime import datetime
from tqdm import tqdm
import requests
from time import sleep
import matplotlib.pyplot as plt
import random

importlib.reload(sybil_functions)

from sybil_functions import (
    read_from_multiple_csv,
    check_df,
    count_pairs,
    remove_contract_transactions,
    stretched_sigmoid,
    get_weight_df,
    plot_weight_dist,
    plot_weight_cumulative_dist,
    create_community,
    community_visualization,
    file_name_lst,
    random_rate
)

Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'bayanpy', 'graph_tool', 'infomap', 'leidenalg'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw', 'pyclustering'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'leidenalg', 'wurlitzer', 'infomap'}


# Import data

## Airdrop wallets

In [3]:
edu_airdrop = pd.read_json(
    r"/Users/floras/Desktop/flora/personal/unique-user-identification/data/EDU_airdrop.json"
)
edu_airdrop = pd.DataFrame(edu_airdrop)
edu_airdrop = edu_airdrop[edu_airdrop['percentageToClaim'] >= 0.5]

In [4]:
edu_airdrop_wallet = edu_airdrop["walletAddress"]
len(edu_airdrop_wallet)


2380

In [5]:
edu_airdrop_wallet = edu_airdrop_wallet.str.lower()
edu_airdrop_wallet = edu_airdrop_wallet.tolist()

## Token transfers

In [6]:
token_transfer_path = r"/Users/floras/Desktop/flora/personal/unique-user-identification/data/edu_airdrop_wallet_token_transfers"
token_transfer_files_lst = file_name_lst(token_transfer_path, startswith="bsc")

In [7]:
token_transfer_df = read_from_multiple_csv(token_transfer_path, token_transfer_files_lst)
token_transfer_df.sort_values(by="block_timestamp", inplace=True)

In [8]:
check_df(token_transfer_df)

(829110, 10)

token_address       object
from_address        object
to_address          object
value               object
transaction_hash    object
log_index            int64
block_timestamp     object
block_number         int64
block_hash          object
amount_raw          object
dtype: object

,log_index,block_number
count,829110.000000,8.291100e+05
mean,385.444814,2.415076e+07
std,734.724967,5.840648e+06
min,0.000000,1.396978e+07
25%,141.000000,1.869658e+07
50%,256.000000,2.524435e+07
75%,444.000000,2.875184e+07
max,55709.000000,3.484866e+07


,token_address,from_address,to_address,value,transaction_hash,log_index,block_timestamp,block_number,block_hash,amount_raw
74456,0xe9e7cea3dedca5984780bafc599bd69add087d56,0x58f876857a02d6762e0101bb5c46a8c1ed44dc16,0xd512ffe85be6f9ce44091c4bfb474f0293e818dc,8616002343588278118,0x156998ee622984a99f9f86c6bc14a4837c91e8bac518...,484,2022-01-01 00:05:52+00:00,13969778,0x6e3d15c5a1aa00d909e259a24cc3a8d181824641093b...,8616002343588278118
97000,0x267022751e06d97b9ee4e5f26cc1023670bdb349,0xd512ffe85be6f9ce44091c4bfb474f0293e818dc,0x1c183b23e8d7a8d287606364f741ddbdafec28de,10615596330275200000,0x156998ee622984a99f9f86c6bc14a4837c91e8bac518...,479,2022-01-01 00:05:52+00:00,13969778,0x6e3d15c5a1aa00d909e259a24cc3a8d181824641093b...,10615596330275200000
97777,0xcb2b25e783a414f0d20a65afa741c51b1ad84c49,0x3fbec4e71bfca562a1631b3870ad8d79623ca2c3,0xea9f1bd192c06f3b594062231721edbb5357806d,2608000000000000000000,0x2ecd2432c32a203aabe02526f7c13062c1eb4cc972e2...,208,2022-01-01 00:06:34+00:00,13969792,0xfa4589d9bc79aee76db839324cd4f9357e5a2815b565...,2608000000000000000000
107244,0xaf307c4ea25c8d2a7029dc0322396a6b4b00a0fc,0x056f3336dc183d27fe6785bae037fdb48c77bc71,0x4cd279347bd15cca417439064ee24f5ed3391a8e,14109370034,0x6830c1b1e77b7a5c99bff8d07c317239284268b3f750...,170,2022-01-01 00:08:22+00:00,13969828,0x046aca96814ecd1c7fa63c61ca360cad265f033abb46...,14109370034
107242,0xaf307c4ea25c8d2a7029dc0322396a6b4b00a0fc,0x056f3336dc183d27fe6785bae037fdb48c77bc71,0x4cd279347bd15cca417439064ee24f5ed3391a8e,500000000000,0x0667e8770dfaa72d783c0f4c6fa6ba145be0f1f04e6d...,378,2022-01-01 00:08:43+00:00,13969835,0xcc9604ce294468e5f95102dcf6c46b19dc0b48dd2f6f...,500000000000


## Native token transfers

In [9]:
native_token_transfer_path = r"/Users/floras/Desktop/flora/personal/unique-user-identification/data/edu_airdrop_wallet_native_token_transfers"
native_token_transfer_file_lst = file_name_lst(
    native_token_transfer_path,
    startswith="bsc_native_token_transfers"
)
native_token_transfer_df = read_from_multiple_csv(
    native_token_transfer_path, native_token_transfer_file_lst
)
native_token_transfer_df.sort_values(by="block_timestamp", inplace=True)

In [11]:
check_df(native_token_transfer_df)

(189105, 10)

token_address        object
transaction_hash     object
transaction_index     int64
block_timestamp      object
trace_id             object
from_address         object
to_address           object
amount_raw           object
call_type            object
block_number          int64
dtype: object

,transaction_index,block_number
count,189105.000000,1.891050e+05
mean,97.476079,2.540913e+07
std,138.107720,6.372803e+06
min,0.000000,1.396987e+07
25%,45.000000,1.995523e+07
50%,79.000000,2.653818e+07
75%,121.000000,3.051036e+07
max,6044.000000,3.484869e+07


,token_address,transaction_hash,transaction_index,block_timestamp,trace_id,from_address,to_address,amount_raw,call_type,block_number
27725,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0x135f41d360be2a3c5b8a43dc23970a4edb53f1cb75d2...,43,2022-01-01 00:10:25+00:00,call_0x135f41d360be2a3c5b8a43dc23970a4edb53f1c...,0x4cd279347bd15cca417439064ee24f5ed3391a8e,0x10ed43c718714eb63d5aa57b78b54704e256024e,6183914692940477,call,13969869
26910,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0x7a692e331b6174701fe663b8d39025b0aeb2561e51d5...,9,2022-01-01 00:12:13+00:00,call_0x7a692e331b6174701fe663b8d39025b0aeb2561...,0x0d0707963952f2fba59dd06f2b425ace40b492fe,0x60e81aac6ce31d84121732362c9855ae1a9f09f3,10994100000000000000,call,13969905
24328,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0x1fce037731562e0f5e098c46677383f815602c8753c2...,113,2022-01-01 00:13:16+00:00,call_0x1fce037731562e0f5e098c46677383f815602c8...,0x97300791462d8acc2b7882726c2dcafe67a84e2d,0x96f5ac215089e9b9536ba58d5ce7d41ad35a0b75,64588000000000000,call,13969926
26919,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0x2b1894a531984d150c873f5825539d51abef73109d70...,25,2022-01-01 00:13:37+00:00,call_0x2b1894a531984d150c873f5825539d51abef731...,0x60e81aac6ce31d84121732362c9855ae1a9f09f3,0x1111111254fb6c44bac0bed2854e76f90643097d,1000000000000000000,call,13969933
25910,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0x6da9acaffa015ed865c84a2529946ee24b7641cb2857...,18,2022-01-01 00:13:52+00:00,call_0x6da9acaffa015ed865c84a2529946ee24b7641c...,0x60e81aac6ce31d84121732362c9855ae1a9f09f3,0x1111111254fb6c44bac0bed2854e76f90643097d,1000000000000000000,call,13969938


# Set samples

# Before airdrop

In [ ]:
before_token_df = token_transfer_df.copy()

# set date